In [46]:
import zipfile
zip_ref=zipfile.ZipFile('Chinook_Sqlite.sqlite.zip')
zip_ref.extractall()
zip_ref.close()

In [47]:
import sqlite3
import pandas as pd

In [48]:
conn=sqlite3.connect('Chinook_Sqlite.sqlite')

In [49]:
query="""
SELECT * FROM Album
LIMIT 5
"""
df=pd.read_sql(query,conn)
df

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [50]:
query="""
SELECT * FROM Customer
LIMIT 5;
"""
df=pd.read_sql(query,conn)
df

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [51]:
query="""
SELECT SUM(UnitPrice*Quantity) AS Revenue
FROM InvoiceLine
"""
df=pd.read_sql(query,conn)
df

,Revenue
0,2328.6


In [52]:
query="""
SELECT SUM(Quantity) AS TotalQuantity
FROM InvoiceLine
"""
df=pd.read_sql(query,conn)
df

,TotalQuantity
0,2240


In [53]:
query="""
SELECT Album.Title AS AlbumName, Artist.Name AS ArtistName, Track.Name AS TrackName, Genre.Name AS GenreName
FROM Album
JOIN Artist ON Artist.ArtistId=Album.ArtistId
JOIN Track ON Track.AlbumId=Album.AlbumId
JOIN Genre ON Genre.GenreId=Track.GenreId
LIMIT 5;
"""
df=pd.read_sql(query,conn)
df

,AlbumName,ArtistName,TrackName,GenreName
0,For Those About To Rock We Salute You,AC/DC,For Those About To Rock (We Salute You),Rock
1,Balls to the Wall,Accept,Balls to the Wall,Rock
2,Restless and Wild,Accept,Fast As a Shark,Rock
3,Restless and Wild,Accept,Restless and Wild,Rock
4,Restless and Wild,Accept,Princess of the Dawn,Rock


Top 5 best selling product


In [54]:
query="""
SELECT Track.Name AS Product , SUM(InvoiceLine.Quantity) AS TotalSold
FROM InvoiceLine
JOIN Track ON Track.TrackId=InvoiceLine.TrackId
GROUP BY Track.TrackId
ORDER BY TotalSold DESC
LIMIT 5;
"""
df=pd.read_sql(query,conn)
df

,Product,TotalSold
0,Balls to the Wall,2
1,Inject The Venom,2
2,Snowballed,2
3,Overdose,2
4,Deuces Are Wild,2


In [55]:
query="""
SELECT Artist.Name AS ArtistName, Album.Title AS AlbumName
FROM Album
JOIN Artist ON Artist.ArtistId=Album.ArtistId
ORDER BY ArtistName DESC
LIMIT 5;
"""
df=pd.read_sql(query,conn)
df

,ArtistName,AlbumName
0,Zeca Pagodinho,Ao Vivo [IMPORT]
1,Yo-Yo Ma,Bach: The Cello Suites
2,Yehudi Menuhin,Bartok: Violin & Viola Concertos
3,Wilhelm Kempff,Bach: Goldberg Variations
4,Vinícius De Moraes,Vinicius De Moraes


Revenue By Country


In [56]:
query="""
SELECT Customer.Country, SUM(InvoiceLine.Quantity*InvoiceLine.UnitPrice) AS TotalRevenue
FROM InvoiceLine
JOIN Invoice ON Invoice.InvoiceId=InvoiceLine.InvoiceId
JOIN Customer ON Customer.CustomerId=Invoice.CustomerId
GROUP BY Customer.Country
ORDER BY TotalRevenue DESC
LIMIT 5;
"""
df=pd.read_sql(query,conn)
df

,Country,TotalRevenue
0,USA,523.06
1,Canada,303.96
2,France,195.10
3,Brazil,190.10
4,Germany,156.48


Revenue By City


In [57]:
query="""
SELECT Customer.City , SUM(InvoiceLine.Quantity*InvoiceLine.UnitPrice) AS TotalRevenue
FROM InvoiceLine
JOIN Invoice ON Invoice.InvoiceId=InvoiceLine.InvoiceId
JOIN Customer ON Customer.CustomerId=Invoice.CustomerId
GROUP BY Customer.City
ORDER BY TotalRevenue DESC
LIMIT 5;
"""
df=pd.read_sql(query,conn)
df

,City,TotalRevenue
0,Prague,90.24
1,Paris,77.24
2,Mountain View,77.24
3,São Paulo,75.24
4,London,75.24


In [58]:
query="""
SELECT strftime('%Y-%m-%d',Invoice.InvoiceDate) AS Date, SUM(InvoiceLine.UnitPrice*InvoiceLine.Quantity) AS Revenue
FROM Invoice
JOIN InvoiceLine ON InvoiceLine.InvoiceId=Invoice.InvoiceId
GROUP BY Date
ORDER BY Revenue DESC
LIMIT 5;
"""
df=pd.read_sql(query,conn)
df

,Date,Revenue
0,2013-11-13,25.86
1,2012-08-05,23.86
2,2010-02-18,21.86
3,2011-04-28,21.86
4,2011-05-29,18.86


In [59]:
query="""SELECT
    Employee.FirstName || ' ' || Employee.LastName AS EmployeeName,
    COUNT(Customer.CustomerId) AS TotalCustomers
FROM Customer
JOIN Employee ON Customer.SupportRepId = Employee.EmployeeId
GROUP BY Employee.EmployeeId
"""
df=pd.read_sql(query,conn)
df

,EmployeeName,TotalCustomers
0,Jane Peacock,21
1,Margaret Park,20
2,Steve Johnson,18


In [60]:
query="""
SELECT Customer.FirstName || ' '|| Customer.LastName AS CustomerName, Track.Name AS TrackName, Artist.Name AS ArtistName,
Customer.Country,
Invoice.InvoiceDate,
InvoiceLine.Quantity,
InvoiceLine.UnitPrice,
InvoiceLine.Quantity*InvoiceLine.UnitPrice AS Total
FROM Invoice
JOIN Customer ON Customer.CustomerId=Invoice.CustomerId
JOIN Artist ON Artist.ArtistId=Album.ArtistId
JOIN Track ON Track.TrackId=InvoiceLine.TrackId
JOIN Album ON Album.AlbumId=Track.AlbumId
JOIN InvoiceLine ON InvoiceLine.InvoiceId=Invoice.InvoiceId
LIMIT 10;
"""
df=pd.read_sql(query,conn)
df

,CustomerName,TrackName,ArtistName,Country,InvoiceDate,Quantity,UnitPrice,Total
0,Leonie Köhler,Balls to the Wall,Accept,Germany,2009-01-01 00:00:00,1,0.99,0.99
1,Leonie Köhler,Restless and Wild,Accept,Germany,2009-01-01 00:00:00,1,0.99,0.99
2,Bjørn Hansen,Put The Finger On You,AC/DC,Norway,2009-01-02 00:00:00,1,0.99,0.99
3,Bjørn Hansen,Inject The Venom,AC/DC,Norway,2009-01-02 00:00:00,1,0.99,0.99
4,Bjørn Hansen,Evil Walks,AC/DC,Norway,2009-01-02 00:00:00,1,0.99,0.99
5,Bjørn Hansen,Breaking The Rules,AC/DC,Norway,2009-01-02 00:00:00,1,0.99,0.99
6,Daan Peeters,Dog Eat Dog,AC/DC,Belgium,2009-01-03 00:00:00,1,0.99,0.99
7,Daan Peeters,Overdose,AC/DC,Belgium,2009-01-03 00:00:00,1,0.99,0.99
8,Daan Peeters,Love In An Elevator,Aerosmith,Belgium,2009-01-03 00:00:00,1,0.99,0.99
9,Daan Peeters,Janie's Got A Gun,Aerosmith,Belgium,2009-01-03 00:00:00,1,0.99,0.99
